# Monte Carlo Control

In [ ]:
import sys
import gym
import numpy as np
from collections import defaultdict
from plot_utils import plot_blackjack_values, plot_policy

In [ ]:
env = gym.make('Blackjack-v1')

In [ ]:
"""
    Get the probability of taking the best known action according to epsilon.
    Returns the policy for the Q value given
"""
def get_probs(Q_s, epsilon, nA):
    policy_s = np.ones(nA) * epsilon / nA
    best_a = np.argmax(Q_s)
    policy_s[best_a] = 1 - epsilon + (epsilon / nA)
    return policy_s

 """
    returns the best actions for each Q value in the policy
    """
def best_policy(Q):
    return dict((k,np.argmax(v)) for k, v in Q.items())
"""
    Creates a greedy policy based on Q values.
    
    Args:
        Q: A dictionary that maps from state -> action values
        
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities.
"""
def create_greedy_policy(Q):   
    def policy_fn(state):
        A = np.zeros_like(Q[state], dtype=float)
        best_action = np.argmax(Q[state])
        A[best_action] = 1.0
        return A
    return policy_fn

def epsilon_greedy(Q, epsilon, nA):
    if np.random.rand() < epsilon:
        return np.random.choice(nA)  # Choose a random action with probability epsilon
    else:
        return np.argmax(Q)  # Choose the action with the highest estimated value according to Q

"""
    Creates a random policy function.
    
    Args:
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities
    """
def create_random_policy(nA): # Policy where all actions are equally likely 
    A = np.ones(nA, dtype=float) / nA
    def policy_fn():
        return A
    return policy_fn

In [ ]:
"""
    Calculate the new Q values for the actions taken in the given episode.
    Returns the new Q policy
""" 

def update_Q_first_visit(episode, Q, alpha, gamma):
    visited = set()  # Keep track of visited state-action pairs
    for t, (state, action, reward) in enumerate(episode):
        if (state, action) not in visited:
            G = sum([x[2]*(gamma**i) for i,x in enumerate(episode[t:])])
            Q[state][action] = Q[state][action] + alpha*(G - Q[state][action])
            visited.add((state, action))
    return Q

def update_Q_every_visit(episode, Q, alpha, gamma):
    for t, (state, action, reward) in enumerate(episode):
        G = sum([x[2]*(gamma**i) for i,x in enumerate(episode[t:])])
        Q[state][action] = Q[state][action] + alpha*(G - Q[state][action])
    return Q

def update_Q_wis(C, Q, behavior_policy, episode, gamma, states, target_policy):
    # Sum of discounted returns
    G = 0.0
    # The importance sampling ratio (the weights of the returns)
    weight_ratio = 1.0
    # For each step in the episode, backwards
    for t in range(len(episode))[::-1]:
        state, action, reward = episode[t]
        states.add(state)
        # Update the total reward since step t
        G = gamma * G + reward
        # Update weighted importance sampling formula denominator
        C[state][action] += weight_ratio
        # This also improves our target policy which holds a reference to Q
        Q[state][action] += (weight_ratio / C[state][action]) * (G - Q[state][action])
        # If the action taken by the behavior policy is not the action 
        # taken by the target policy the probability will be 0 and we can break
        if action != np.argmax(target_policy(state)):
            break
        weight_ratio = weight_ratio * 1. / behavior_policy(state)[action]
    return Q,C,states

In [ ]:
def play_game_first_visit(env, Q, epsilon, nA):
    episode = []
    state = env.reset()
    visited = set()  # Keep track of visited state-action pairs
    terminated = False
    while not terminated:
        if type(state[0]) == tuple:
            state = state[0]
        action = epsilon_greedy(Q[state], epsilon, nA)
        next_state, reward, terminated, truncated,info = env.step(action)
        if (state, action) not in visited:
            episode.append((state, action, reward))
        visited.add((state, action))  # Mark this state-action pair as visited
        state = next_state    
    return episode

def play_game_every_visit(env, Q, epsilon, nA):
    episode = []
    state = env.reset()
    terminated = False
    while not terminated:
        if type(state[0]) == tuple:
            state = state[0]
        action = epsilon_greedy(Q[state], epsilon, nA)
        next_state, reward, terminated, truncated,info  = env.step(action)
        episode.append((state, action, reward))
        
        state = next_state    
    return episode

def play_game_wis(behavior_policy, env):
    episode = []
    state = env.reset()
    terminated = False
    while not terminated:
        if type(state[0]) == tuple:
            state = state[0]
        probs = behavior_policy(state)
        action = np.random.choice(np.arange(len(probs)), p=probs)
        next_state, reward, terminated, trunc, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
    return episode

In [ ]:
 """
    main method. Iterates through episodes updating epsilon after each, retrieves the list of states, actions
    and rewards from the last episode and use them to calculate the updated Q values
    """
def mc_control(env, num_episodes,first_visit=True,gamma=.8): 
    epsilon = .8
    eps_min = 0.01
    decay = 0.99
    alpha = 0.001
    
    nA = env.action_space.n
    Q = defaultdict(lambda: np.zeros(nA))
    for i_episode in range(1, num_episodes+1):
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        epsilon = max(epsilon*decay, eps_min)
        if first_visit:
            episode = play_game_first_visit(env, Q, epsilon, nA)
            Q = update_Q_first_visit(episode, Q, alpha, gamma)
        else:
            episode = play_game_every_visit(env, Q, epsilon, nA)
            Q = update_Q_every_visit(episode, Q, alpha, gamma)
    policy = best_policy(Q)
    return policy, Q

"""
    Monte Carlo Control Off-Policy Control using Weighted Importance Sampling.
    Finds an optimal greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        behavior_policy: The behavior to follow while generating episodes.
            A function that given an observation returns a vector of probabilities for each action.
        discount_factor: Gamma discount factor.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities. This is the optimal greedy policy.
    """
def mc_control_importance_sampling(env, num_episodes, behavior_policy, gamma =.8):
    
    # The final action-value function.
    nA = env.action_space.n
    Q = defaultdict(lambda: np.zeros(nA))
    # The cumulative denominator of the weighted importance sampling formula,across all episodes
    C = defaultdict(lambda: np.zeros(nA))
    states = set()
    # Our greedily policy we want to learn
    target_policy = create_greedy_policy(Q)
        
    for i_episode in range(1, num_episodes + 1):
        # Print out which episode we're on
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        # Generate an episode.
        episode = play_game_wis(behavior_policy, env)
        Q,C,states = update_Q_wis(C, Q, behavior_policy, episode, gamma, states, target_policy)

    return target_policy, Q,states

In [ ]:
def launch_monte_carlo(nb_episodes, first_visit=True):
    policy, Q = mc_control(env, nb_episodes, first_visit)
    V = dict((k,np.max(v)) for k, v in Q.items())
    plot_blackjack_values(V)
    new_policy = {}
    for key,value in policy.items():
        new_policy[(key[1],key[0],key[2])] = value
    plot_policy(new_policy)

In [ ]:
launch_monte_carlo(500000)

In [ ]:
launch_monte_carlo(500000, False)

In [ ]:
def display_policy_wis(policy_function):
    weighted_policy = {(i[1], i[0], i[2]): np.argmax(policy_function(i)) for i in states_weighted}
    plot_policy(weighted_policy)

In [ ]:
random_policy = create_random_policy(env.action_space.n)

def launch_monte_carlo_wis(behavior_policy,nb_episodes=500000, gamma=.8):
    global states_weighted
    policy_weighted, Q_weighted, states_weighted = mc_control_importance_sampling(env, nb_episodes, behavior_policy,gamma)
    V_weigthed = dict((k, np.max(v)) for k, v in Q_weighted.items())
    plot_blackjack_values(V_weigthed)
    display_policy_wis(policy_weighted)

launch_monte_carlo_wis(random_policy)